In [5]:
!pip install pandas numpy yfinance

  Using cached pandas-2.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.1-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached yfinance-0.2.63-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.6-py313-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.1-cp313-cp313-macosx_15_0_arm64.whl
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached curl_cffi-0.11.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (14 kB)
  Using cached protobuf-6.31.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.

In [6]:
import pandas as pd
import numpy as np
import time
import yfinance as yf

In [7]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [8]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]
    
    #time.sleep(0.5)

HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 


In [11]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,93588653,603516960768,The trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,NaN,NaN,NaN
EEM,iShares MSCI Emerging Index Fun,ETF,USD,32462175,17162833920,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,NaN,NaN,NaN
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,21639,696944896,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,44876836,15944028160,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,989890,1142769792,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,NaN,NaN,NaN
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,NaN,NaN,NaN


In [12]:
STARTDATE = '2011-01-01'
ENDDATE = '2025-05-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']

/var/folders/5y/9gbrlsgx0sn17yw7ydqrny4h0000gn/T/ipykernel_87833/1766415481.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']
[*********************100%***********************]  12 of 12 completed


In [15]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()
rets['SHV'] -= .01/20

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/5y/9gbrlsgx0sn17yw7ydqrny4h0000gn/T/ipykernel_87833/1814017950.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [18]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2011-02-28,0.006617,0.041710,-0.000437,0.035498,0.014355,-0.002084,0.045203,0.040146,0.002583,-0.000409,0.034737,0.007208
2011-03-31,0.008427,0.026582,0.062896,-0.023883,0.000425,-0.001546,-0.010934,0.016667,0.005521,-0.000328,0.000120,0.011903
2011-04-30,0.048805,0.045559,0.027327,0.056258,0.015975,0.018413,0.046632,0.058671,0.019034,0.000044,0.028962,0.023780
2011-05-31,-0.013546,-0.051724,-0.029400,-0.022061,0.001333,0.025056,0.010134,-0.041565,0.000000,-0.000600,-0.011214,0.002659
2011-06-30,-0.000097,-0.042645,-0.009355,-0.012107,-0.005706,-0.005044,-0.031060,-0.042581,-0.010776,-0.000328,-0.016870,0.007549
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,0.000935,0.027596,0.021521,0.048009,0.013605,0.006166,0.018913,0.067995,0.016237,0.003133,0.026856,0.013703
2025-02-28,0.013991,0.001366,0.011470,0.029531,0.009703,0.027997,0.037861,-0.040107,-0.005952,0.002615,-0.012695,0.021572
2025-03-31,0.010353,0.022727,0.011340,0.001839,-0.010858,0.003419,-0.023382,-0.060826,-0.013237,0.002843,-0.055719,0.006797


In [19]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.007026,0.042120,-0.000027,0.035907,0.014764,-0.001675,0.045612,0.040555,0.002992,0.035147,0.007617
2011-03-31,0.008755,0.026909,0.063224,-0.023555,0.000752,-0.001218,-0.010607,0.016994,0.005848,0.000448,0.012231
2011-04-30,0.048761,0.045515,0.027283,0.056214,0.015931,0.018369,0.046588,0.058627,0.018990,0.028918,0.023736
2011-05-31,-0.012946,-0.051124,-0.028800,-0.021461,0.001933,0.025656,0.010733,-0.040965,0.000600,-0.010615,0.003259
2011-06-30,0.000231,-0.042317,-0.009027,-0.011780,-0.005379,-0.004716,-0.030732,-0.042253,-0.010448,-0.016542,0.007877
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,-0.002198,0.024463,0.018388,0.044877,0.010472,0.003033,0.015780,0.064862,0.013104,0.023723,0.010570
2025-02-28,0.011376,-0.001250,0.008855,0.026916,0.007087,0.025382,0.035246,-0.042722,-0.008567,-0.015310,0.018957
2025-03-31,0.007510,0.019885,0.008497,-0.001004,-0.013701,0.000577,-0.026225,-0.063669,-0.016079,-0.058562,0.003955


In [20]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')

ModuleNotFoundError: No module named 'openpyxl'